### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

In [1]:
#import modules for analysis

import pandas as pd
import numpy as np

#Retrieve data

csvpath = ("Resources/purchase_data.csv")

purchase_data = pd.read_csv(csvpath)

purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [2]:
#find total unique players since one could make multiple purchases

demographic_df = purchase_data.loc[:,["SN","Age", "Gender"]]

demographic_df = demographic_df.drop_duplicates()
unique_players = demographic_df["SN"].count()

summary_players = pd.DataFrame({"Total Players":[unique_players]})

summary_players

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
purchase_analysis = purchase_data.loc[:,["Item ID","Item Name","Price"]]

average_price = purchase_analysis["Price"].mean()
unique_items = len(purchase_analysis["Item ID"].unique())
purchase_count = purchase_analysis["Item ID"].count()
purchase_total = purchase_analysis["Price"].sum()

purchase_analysis_summary = pd.DataFrame({"Unique Items":unique_items,"Purchase Count":[purchase_count],
                                          "Average Price":[average_price], "Total Revenue":[purchase_total]})

def format_string(x):
    return f"${x:,.2f}"
    
purchase_analysis_summary["Average Price"] = purchase_analysis_summary["Average Price"].map(format_string)
purchase_analysis_summary["Total Revenue"] = purchase_analysis_summary["Total Revenue"].map(format_string)

purchase_analysis_summary

,Unique Items,Purchase Count,Average Price,Total Revenue
0,183,780,$3.05,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
gen_count = demographic_df["Gender"].value_counts()
percentage_gender = ((gen_count/ summary_players.iloc[0,0])*100).round(2)

summary_gen_df = pd.concat([gen_count, percentage_gender], axis=1)
summary_gen_df = summary_gen_df.set_axis(["Total Players", "Percentage"], axis=1)

summary_gen_df

,Total Players,Percentage
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
purchase_analysis_gen = purchase_data.loc[:,["Purchase ID", "SN", "Gender", "Price"]]

purchase_count = purchase_analysis_gen[["Purchase ID", "Gender"]].groupby(["Gender"]).count().rename(columns={"Purchase ID":"Total Purchases"})
avg_purchase_price = purchase_analysis_gen[["Price", "Gender"]].groupby(["Gender"]).mean().rename(columns={"Price": "Average Price"})
total_purchase_value = purchase_analysis_gen[["Price", "Gender"]].groupby(["Gender"]).sum().rename(columns={"Price": "Total Revenue"})
avg_number_purchases = (total_purchase_value["Total Revenue"] / summary_gen_df["Total Players"]).round(2)

summary_purchase_analysis = pd.concat([purchase_count, avg_purchase_price, total_purchase_value, avg_number_purchases], axis=1)

summary_purchase_analysis = summary_purchase_analysis.rename(columns={0:"Average Total Purchase"})
summary_purchase_analysis["Average Price"] = summary_purchase_analysis["Average Price"].map(format_string)
summary_purchase_analysis["Total Revenue"] = summary_purchase_analysis["Total Revenue"].map(format_string) 
summary_purchase_analysis["Average Total Purchase"] = summary_purchase_analysis["Average Total Purchase"].map(format_string)  
summary_purchase_analysis

,Total Purchases,Average Price,Total Revenue,Average Total Purchase
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins.


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Round the percentage column to two decimal points


* Display Age Demographics Table


In [6]:
bins = [0, 9, 14, 19, 24, 29, 34, 39, 150]
names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

demographic_df["Age Group"] = pd.cut(demographic_df["Age"], bins, labels = names)

age_bin_total = demographic_df[["Age", "Age Group"]].groupby(["Age Group"]).count()
age_bin_total = age_bin_total.rename(columns={"Age": "Age Count"})
age_bin_sum = age_bin_total["Age Count"].sum()
age_bin_total["Percentage"] = ((age_bin_total["Age Count"] / age_bin_sum)*100).round(2)

age_bin_total


,Age Count,Percentage
Age Group,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
purchase_age_df = purchase_data.loc[:,["Age","Item ID","Item Name","Price",]]

purchase_age_df["Age Group"] = pd.cut(purchase_age_df["Age"], bins, labels = names)

purchase_age_group = purchase_age_df.groupby(["Age Group"])

purchase_count_age = purchase_age_group[["Price"]].count().rename(columns={"Price": "Purchase Count"})
avg_purchase_price_age = purchase_age_group[["Price"]].mean().rename(columns={"Price": "Average Purchase Price"})
sum_of_purchase = purchase_age_group[["Price"]].sum().rename(columns={"Price": "Total"})
avg_purchase_total_age = (sum_of_purchase["Total"] / age_bin_total["Age Count"])

summary_purchase_age = pd.concat([purchase_count_age,avg_purchase_price_age,sum_of_purchase,avg_purchase_total_age], axis=1)

summary_purchase_age = summary_purchase_age.rename(columns={0:"Average Purchase Total"})
summary_purchase_age["Average Purchase Price"] = summary_purchase_age["Average Purchase Price"].map(format_string)
summary_purchase_age["Total"] = summary_purchase_age["Total"].map(format_string)
summary_purchase_age["Average Purchase Total"] = summary_purchase_age["Average Purchase Total"].map(format_string)
summary_purchase_age

,Purchase Count,Average Purchase Price,Total,Average Purchase Total
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [8]:
spend_purchase_df = purchase_data.loc[:,["SN","Price",]]

spend_group = spend_purchase_df.groupby(["SN"])
spend_purchase_count = spend_group.count().rename(columns={"Price": "Total Count"})
spend_avg_purchase_price = spend_group.mean().rename(columns={"Price": "Average Purchase Price"})
spend_total_purchase_value = spend_group.sum().rename(columns={"Price": "Total Purchase Value"})

top_spend_summary = pd.concat([spend_purchase_count,spend_avg_purchase_price, spend_total_purchase_value], axis=1)

top_spend_summary = top_spend_summary.sort_values("Total Purchase Value", ascending=False)

top_spend_summary["Average Purchase Price"] = top_spend_summary["Average Purchase Price"].map(format_string)
top_spend_summary["Total Purchase Value"] = top_spend_summary["Total Purchase Value"].map(format_string)

top_spend_summary.head()

,Total Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.41,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
most_pop_df = purchase_data.loc[:,["Item ID","Item Name", "Price"]]
most_pop_group = most_pop_df.groupby(["Item ID", "Item Name"])
item_purchase_count = most_pop_group.count().rename(columns={"Price":"Purchase Count"})
item_price = most_pop_group.mean().rename(columns={"Price":"Item Price"})
pop_total = most_pop_group.sum().rename(columns={"Price":"Total Purchase Value"})

pop_summary = pd.concat([item_purchase_count,item_price, pop_total], axis=1)
pop_summary = pop_summary.sort_values("Purchase Count", ascending=False)

pop_summary["Item Price"] = pop_summary["Item Price"].map(format_string)
pop_summary["Total Purchase Value"] = pop_summary["Total Purchase Value"].map(format_string)
pop_summary.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Give the displayed data cleaner formatting


* Display a preview of the data frame



In [10]:
pop_summary = pd.concat([item_purchase_count,item_price, pop_total], axis=1)

pop_summary = pop_summary.sort_values("Total Purchase Value", ascending=False)

pop_summary["Item Price"] = pop_summary["Item Price"].map(format_string)
pop_summary["Total Purchase Value"] = pop_summary["Total Purchase Value"].map(format_string)

pop_summary.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
